In [2]:
import numpy as np
# import pathogenprofiler as pp
from sklearn.mixture import GaussianMixture
# import fastq2matrix as fm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from scipy.stats import norm
import subprocess


In [11]:
vcf_file = 'test_data/ERR6634978-ERR6635032-4060.vcf.gz'
json_file = 'test_data/ERR6635386.results.json'

In [12]:
def plot_gm(gm,data):
    gm.covariances_[0][0][0]
    std0 = np.sqrt(gm.covariances_[0][0][0]) #(n_components, n_features, n_features)
    mu0 = gm.means_[0][0]
    std1 = np.sqrt(gm.covariances_[1][0][0]) #component 1?
    mu1 = gm.means_[1][0]
    x = np.linspace(0, 1, 100)
    p0 = norm.pdf(x, mu0, std0)
    p1 = norm.pdf(x, mu1, std1)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Histogram(x=[x[0] for x in data]),secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=p0, mode='lines'),secondary_y=True)
    fig.add_trace(go.Scatter(x=x, y=p1, mode='lines'),secondary_y=True)
    fig.show()

In [26]:
def vcf_to_mix_model(vcf_file,plot=False,tail_cutoff=0.05,title="AF Histogram",return_freqs = False):
    pos = []
    freqs = []
    with open('mix_infection.csv', 'w') as f:
        subprocess.run("bcftools view -c 1 -m2 -M2 -T ^new_exclusion.bed %s | bcftools query -f '%%POS\\t%%REF\\t%%ALT[\\t%%GT\\t%%AD\\n]'" % vcf_file, shell=True, stdout=f, text=True)

#count how many column there is in the ROAO_proportion.csv file this is needed in order to read the csv in a a panda dataframe

    with open('mix_infection.csv', 'r') as f:
        for l in f:
            row = l.strip().split()
            ads = [int(x) for x in row[4].split(",")]
            afs = [x/sum(ads) for x in ads]
            if afs[1]>1-tail_cutoff or afs[1]<tail_cutoff:
                continue
            pos.append(int(row[0]))
            freqs.append([afs[1]])
        # freqs = [[0.7],[0.6],[0.4]]    
    gm = GaussianMixture(n_components=2, random_state=0).fit(freqs)
    if plot:
        plot_gm(gm,freqs)
    if return_freqs:
        return (gm,list(zip(pos,freqs)))
    else:
        return gm



In [27]:
def assign_variant_to_distrib(gm,freq,cutoff=0.95):
    probs = gm.predict_proba([[freq]])
    pred = gm.predict([[freq]])
    if probs[0][pred][0]>cutoff:
        return pred[0]
    else:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        return None

In [28]:
gm = vcf_to_mix_model(vcf_file,plot=True,tail_cutoff=0.2)

In [ ]:
json_results = json.load(open(json_file))
strain0 = []
strain1 = []
strainU = []

for var in json_results['dr_variants']:
    cluster = assign_variant_to_distrib(gm,var['freq'])
    if cluster == 0:
        strain0.append(var)
    elif cluster == 1:
        strain1.append(var)
    else:
        strainU.append(var)

In [ ]:
[(v['gene'],v['change'],v['freq']) for v in strain0]


In [ ]:
[(v['gene'],v['change'],v['freq']) for v in strain1]


In [ ]:
[(v['gene'],v['change'],v['freq']) for v in strainU]
